In [1]:
from datetime import date
import calendar as c
import pandas as pd
import numpy as np

#### Para testear las funciones, levanto los dataframes originales

In [4]:
df_pageviews = pd.read_csv('data/pageviews.csv', parse_dates=['FEC_EVENT'])
year = 2018
mes_snapshot = 9

def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

universo = universo_train(df_pageviews)

In [5]:
def _sum_campo_user(df_pageviews, campo):
    temp = pd.crosstab(df_pageviews.USER_ID, df_pageviews[campo]) 
    temp.columns = [campo + "_" + str(v) for v in temp.columns]
    return temp

In [6]:
def _sum_campo_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_campo_user(df, campo)

In [7]:
def _sum_total_user(df_pageviews, campo):
    return df_pageviews.groupby("USER_ID")[campo].sum()

In [8]:
def _sum_total_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_total_user(df, campo)

In [9]:
def _sum_user_campo_all(df_pageviews, mes_snapshot, campo):
    fecha_desde_1m = pd.Timestamp(year, mes_snapshot, 1)
    fecha_desde_15d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 15)
    fecha_desde_7d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 7)
    fecha_desde_1d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 1)
    fecha_hasta_last_day = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    
    df_1m = _sum_campo_user_between(df_pageviews, fecha_desde_1m, fecha_hasta_last_day, campo)
    df_15d = _sum_campo_user_between(df_pageviews, fecha_desde_15d, fecha_hasta_last_day, campo)
    df_7d = _sum_campo_user_between(df_pageviews, fecha_desde_7d, fecha_hasta_last_day, campo)
    df_1d =_sum_campo_user_between(df_pageviews, fecha_desde_1d, fecha_hasta_last_day, campo)
    df_ever =_sum_campo_user(df_pageviews, campo)

    tot_1m = _sum_total_user_between(df_pageviews, fecha_desde_1m, fecha_hasta_last_day, campo)
    tot_15d = _sum_total_user_between(df_pageviews, fecha_desde_15d, fecha_hasta_last_day, campo)
    tot_7d = _sum_total_user_between(df_pageviews, fecha_desde_7d, fecha_hasta_last_day, campo)    
    tot_1d = _sum_total_user_between(df_pageviews, fecha_desde_1d, fecha_hasta_last_day, campo)
    tot_ever = _sum_total_user(df_pageviews, campo)

    return df_1m,df_15d, df_7d, df_1d, df_ever, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever

In [112]:
def _ratio_campo_df(df_numerador, df_denominador, name):
    temp = df_numerador.div(df_denominador, axis=0)
    temp.columns = [name + "_" + str(v) for v in temp.columns]
    return temp

In [113]:
def _ratio_campo_series(df_numerador, df_denominador, name):
    temp = df_numerador.div(df_denominador, axis=0).to_frame()
    temp.columns = [name + "_" + str(v) for v in temp.columns]
    return temp

In [122]:
def _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo):
    df_1m,df_15d, df_7d, df_1d, df_ever, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever = _sum_user_campo_all(df_pageviews, mes_snapshot, campo)
    
    ratio_preferencia_1m = _ratio_campo_df(df_1m, tot_1m, 'ratio_preferencia_1m')
    ratio_preferencia_15d = _ratio_campo_df(df_15d, tot_15d, 'ratio_preferencia_15d')
    ratio_preferencia_7d = _ratio_campo_df(df_7d, tot_7d, 'ratio_preferencia_7d')
    ratio_preferencia_1d = _ratio_campo_df(df_1d, tot_1d, 'ratio_preferencia_1d')
    
    ratio_interes_1m = _ratio_campo_df(df_1m, tot_ever, 'ratio_interes_1m')
    ratio_interes_15d = _ratio_campo_df(df_15d, tot_ever, 'ratio_interes_15d')
    ratio_interes_7d = _ratio_campo_df(df_7d, tot_ever, 'ratio_interes_7d')
    ratio_interes_1d = _ratio_campo_df(df_1d, tot_ever, 'ratio_interes_1d')
    
    ratio_interes_reciente_15d = _ratio_campo_df(df_15d, tot_1m, 'ratio_interes_reciente_15d')
    ratio_interes_reciente_7d = _ratio_campo_df(df_7d, tot_1m, 'ratio_interes_reciente_7d')
    ratio_interes_reciente_1d = _ratio_campo_df(df_1d, tot_1m, 'ratio_interes_reciente_1d')

    ratio_actividad_1m = _ratio_campo_series(tot_1m, tot_ever, 'ratio_actividad_1m')
    ratio_actividad_15d = _ratio_campo_series(tot_15d, tot_ever, 'ratio_actividad_15d')
    ratio_actividad_7d = _ratio_campo_series(tot_7d, tot_ever, 'ratio_actividad_7d')
    ratio_actividad_1d = _ratio_campo_series(tot_1d, tot_ever, 'ratio_actividad_1d')
    
    df_union = pd.merge(universo, ratio_preferencia_1m, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_preferencia_15d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_preferencia_7d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_preferencia_1d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    
    df_union = pd.merge(df_union, ratio_interes_1m, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_interes_15d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_interes_7d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_interes_1d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    
    df_union = pd.merge(df_union, ratio_interes_reciente_15d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_interes_reciente_7d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_interes_reciente_1d, how='left', left_on=['user_id'], right_on=['USER_ID'])

    df_union = pd.merge(df_union, ratio_actividad_1m, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_actividad_15d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_actividad_7d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = pd.merge(df_union, ratio_actividad_1d, how='left', left_on=['user_id'], right_on=['USER_ID'])
    
    return df_union

In [123]:
def ratio_page_all(universo, df_pageviews, mes_snapshot):
    campo = 'PAGE'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [106]:
def ratio_ccategory_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [107]:
def ratio_cctop_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_TOP'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [108]:
def ratio_ccbottom_all(univesro, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_BOTTOM'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)